In [119]:
import boto3
import os,sys
import datetime as dt
import glob
import pandas as pd
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
import subprocess as sp

In [120]:
os.makedirs('/dev/shm/logs',exist_ok=True)
os.chdir('/dev/shm/logs')

In [121]:
!aws s3 sync s3://earthbigdata/logs/ /dev/shm/logs --quiet

In [122]:
def make_entry(ip_dict,date_dict,logfile):  
    with open(logfile,"r") as f:
        lines=f.readlines()
    date = lines[0].split('[')[1].split('+')[0].strip()
    date = dt.datetime.strptime(date,'%d/%b/%Y:%H:%M:%S')
    ipaddr=lines[0].split(']')[1].split('-')[0].strip()
    if not ipaddr in ip_dict:
        ip_dict[ipaddr]=[]
    if not date.date() in date_dict:
        date_dict[date.date()]=0
    ip_dict[ipaddr].append(date)
    date_dict[date.date()]+=1
    

In [123]:
ip_dict={}
date_dict={}

for log in glob.glob('ebd-covid19*'):
    make_entry(ip_dict,date_dict,log)

In [124]:
access=pd.DataFrame.from_dict(date_dict,orient='index')

In [125]:
access.sort_index(inplace=True,ascending=False)

In [126]:
access.hvplot.barh(padding=0.1,grid=True,ylabel='Access Times per day',xlabel='',title='EBD COVID 19 Access')

:Bars   [index]   (0)

In [127]:
ip_count={}
for i in ip_dict:
    ip_count[i]=len(ip_dict[i])

In [128]:
access_count=pd.DataFrame.from_dict(ip_count,orient='index')

access_count=access_count[access_count>=5].dropna()

access_count = access_count.sort_values(0,ascending=False)

access_count['nslookup']=''
for i in access_count.index:
    cmd='nslookup {}'.format(i)
    try:
        ret= sp.check_output(cmd.split())
        nslookup = ret.decode().split('name = ')[1].split('\n')[0].strip('.')
        # print(nslookup)
        access_count.loc[i,'nslookup']=nslookup
    except:
        pass

In [129]:
access_count.head(20)

,0,nslookup
24.218.227.219,420.0,c-24-218-227-219.hsd1.ma.comcast.net
52.44.93.197,111.0,ec2-52-44-93-197.compute-1.amazonaws.com
73.23.120.180,96.0,c-73-23-120-180.hsd1.fl.comcast.net
34.231.157.157,91.0,ec2-34-231-157-157.compute-1.amazonaws.com
34.232.127.140,78.0,ec2-34-232-127-140.compute-1.amazonaws.com
72.205.80.22,61.0,ip72-205-80-22.sb.sd.cox.net
93.218.246.5,60.0,p5DDAF605.dip0.t-ipconnect.de
71.232.190.220,60.0,c-71-232-190-220.hsd1.ma.comcast.net
70.191.91.13,51.0,ip70-191-91-13.sb.sd.cox.net
73.143.234.232,50.0,c-73-143-234-232.hsd1.ma.comcast.net


In [130]:
len(list(ip_dict.keys()))

1933

In [131]:
list(ip_dict.keys())[:10]

['195.192.194.133',
 '64.233.172.44',
 '73.23.120.180',
 '178.24.84.37',
 '66.249.83.52',
 '24.218.227.219',
 '64.233.172.46',
 '173.162.212.33',
 '73.143.185.57',
 '73.159.130.33']

In [132]:
for i in list(ip_dict.keys()):
    if ip_dict[i][-1].date() == dt.datetime.now().date():
        if i in list(access_count.index):
            print(ip_dict[i][-1].date(),access_count.loc[i][0],'\t',access_count.loc[i].nslookup)

2020-03-31 5.0 	 23-127-164-61.lightspeed.sntcca.sbcglobal.net
